In [ ]:
import scanpy as sc
import numpy as np

In [ ]:
import rpy2
import rpy2.robjects as robjects

In [ ]:
import anndata2ri
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

In [ ]:
from rpy2.robjects.packages import importr, data

In [ ]:
from openproblems.tasks.cell_cell_communication.metrics.odds_ratio import odds_ratio

## FILES

#### Brain

In [ ]:
adata = sc.read("/media/dbdimitrov/SSDDimitrov/Repos/liana2/data/scop/allen_brain.h5ad")

In [ ]:
del adata.layers['logcounts']

In [ ]:
adata.write_h5ad("/home/dbdimitrov/Downloads/allen_brain.h5ad")

In [ ]:
adata.uns["bench"]

In [ ]:
adata

In [ ]:
sc.pp.subsample(adata, n_obs=500)
adata = adata[:, 30000:30500].copy()

In [ ]:
adata

#### TNBC

In [ ]:
adata = sc.read("/media/dbdimitrov/SSDDimitrov/Repos/liana2/data/scop/brca_tnbc.h5ad")

In [ ]:
del adata.layers['logcounts']

In [ ]:
adata

In [ ]:
adata.write_h5ad("/home/dbdimitrov/Downloads/brca_tnbc.h5ad")

### Test Data LOAD, norm!!!

##### BRAIN

In [ ]:
from openproblems.data.allen_brain_atlas import load_mouse_brain_atlas

In [ ]:
adata = load_mouse_brain_atlas(test=True)

In [ ]:
adata

##### TNBC

In [ ]:
from openproblems.data.tnbc_wu2021 import load_tnbc_data
adata = load_tnbc_data(test=True)
adata

Normalize internally in op

In [ ]:
from openproblems.tools.normalize import log_cpm

In [ ]:
adata = log_cpm(adata)
adata.layers['logcounts'] = adata.layers['log_cpm']
del adata.layers['log_cpm']

In [ ]:
adata

#### TEST method + metric!!!!

### RUN LIANA

In [ ]:
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R -i adata -o adata
library(tidyverse)
library(liana)

method = "natmi"

adata@assays@data$counts <- as(as.matrix(adata@assays@data$logcounts), "sparseMatrix")
adata@assays@data$logcounts <- as(as.matrix(adata@assays@data$counts), "sparseMatrix")


# Here, we will convert LIANA's Consensus resource to murine symbols
op_resource <- select_resource("Consensus")[[1]]

# Check if the target organism is human
if(adata@metadata$target_organism!=9606){
    # Generate orthologous resource
    op_resource <- generate_homologs(op_resource = op_resource,
                                     target_organism = adata@metadata$target_organism)
}


# Run LIANA
adata@metadata$ccc = liana_wrap(adata,
                                resource = 'custom',
                                external_resource = op_resource,
                                method=method,
                                expr_prop = 0.1,
                                idents_col = "label"
                                ) %>%
    # DEAL WITH THIS AT A PER-METHOD BASIS (in R again) ^ code above is shared for all methods
    mutate(score := .data[[liana:::.score_specs()[[method]]@method_score]]) %>%
    arrange(desc(score))

In [ ]:
check_method(adata)

In [ ]:
# Merge liana's results with the truth
gt = adata.uns["bench"].merge(adata.uns["ccc"], how='right')
adata.uns["truth"] = gt[gt['response'].notna()]

In [ ]:
adata.uns["truth"]

In [ ]:
odds_ratio(adata, top_n=1)

In [ ]:
odds_ratio

In [ ]:
import inspect
lines = inspect.getsource(odds_ratio)
print(lines)


In [ ]:
"ccc" in adata.uns

In [ ]:
all( adata.uns["ccc"]

In [ ]:
set(adata.uns["ccc"])

In [ ]:
"response" in adata.uns["bench"]

### Create Sample dataset

In [ ]:
from openproblems.data.sample import load_sample_data

In [ ]:
adata = load_sample_data()

Modify sample data to required input

In [ ]:
# keep only the top 10 most variable
sc.pp.highly_variable_genes(adata, n_top_genes=10)
adata = adata[:,adata.var['highly_variable']]
# assign names to known interactions
adata.var.index = ['LGALS9', 'PTPRC',
                   'LRP1', 'CD47',
                   'CD44', 'COL1A1',
                   'ADAM10', 'SIRPA',
                  'COL4A1', 'THBS2']
# transfer label
adata.obs["label"] = adata.obs.cell_name

# assign to human prior knowledge
adata.uns["target_organism"] = 9606

In [ ]:
adata = log_cpm(adata)
adata.layers['logcounts'] = adata.layers['log_cpm']
del adata.layers['log_cpm']

In [ ]:
adata.uns['ccc']

In [ ]:
method_sample = pd.DataFrame(rng.integers(0, 100, size=(100, 4)), columns=list('ABCD'))

In [ ]:
import pandas as pd
import random

In [ ]:
?random.sample

In [ ]:
df

In [ ]:
df['NEW'] = pd.Series(pd.util.testing.rands(3), index=df.index)

In [ ]:
df

In [ ]:
num_rows = 10

data = np.array([id_generator() for i in range(2*num_rows)]).reshape(-1,2)
pd.DataFrame(data)


Test tests

In [ ]:
from openproblems.tasks.cell_cell_communication.api import check_dataset, sample_dataset, sample_method, check_method

In [ ]:
adata = sample_dataset()

In [ ]:
check_dataset(adata)

In [ ]:
adata = sample_method(adata)

In [ ]:
adata.uns["ccc"]

In [ ]:
check_method(adata)

Test methods

In [1]:
from openproblems.data.tnbc_wu2021 import load_tnbc_data
adata = load_tnbc_data(test=True)
adata

AnnData object with n_obs × n_vars = 500 × 43
    obs: 'label', 'sizeFactor', 'celltype_minor', 'n_counts'
    var: 'n_cells'
    uns: 'bench', 'celltype_major', 'celltype_subset', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'percent.mito', 'subtype', 'target_organism', '_from_cache'
    layers: 'counts'

In [2]:
from openproblems.tasks.cell_cell_communication.methods import logfc, sca

In [3]:
adata = sca(adata)

/home/dbdimitrov/.local/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:182: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata2ri/py2r.py:37: NotConvertedWarning: Conversion 'py2rpy' not defined for objects of type '<class 'NoneType'>'
  warn(str(e), NotConvertedWarning)

    exprs
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.1.0     ✔ forcats 0.5.1
✖ purrr::%@%()         masks rlang::%@%()
✖ purrr::as_function() masks rlang::as_function()
✖ dplyr::collapse()    masks IRanges::collapse()
✖ dplyr::combine()     masks Biobase::combine(), BiocGenerics::combine()
✖ dplyr::count()       masks matrixStats::count()
✖ dplyr::desc()        masks IRanges::desc()
✖ tidyr::expand()      masks Matrix::expand(), S4Vectors::expand()
✖ rlang::exprs()       masks Biobase::exprs()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()  

In [4]:
adata.uns["ccc"]

,source,target,ligand.complex,ligand,receptor.complex,receptor,receptor.prop,ligand.prop,ligand.expr,receptor.expr,global_mean,LRscore,score
1764,Cycling.Myeloid,PVL.Immature,TGFB1,TGFB1,CAV1,CAV1,1.000000,1.000000,12.716901,13.271979,1.06552,0.924200,0.924200
1765,Cycling.Myeloid,Mature.Luminal,TGFB1,TGFB1,ITGB1,ITGB1,1.000000,1.000000,12.716901,13.070524,1.06552,0.923663,0.923663
1738,Cycling.Myeloid,DCs,TGFB1,TGFB1,CXCR4,CXCR4,1.000000,1.000000,12.716901,13.006264,1.06552,0.923489,0.923489
1771,Cycling.Myeloid,Endothelial.CXCL12,TGFB1,TGFB1,CAV1,CAV1,1.000000,1.000000,12.716901,12.981323,1.06552,0.923421,0.923421
1766,Cycling.Myeloid,Mature.Luminal,TGFB1,TGFB1,CXCR4,CXCR4,1.000000,1.000000,12.716901,12.350698,1.06552,0.921641,0.921641
...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,Macrophage,CAFs.MSC.iCAF.like,CXCL12,CXCL12,ITGB3,ITGB3,0.111111,0.108696,1.199687,1.279215,1.06552,0.537602,0.537602
486,Cancer.Basal.SC,CAFs.MSC.iCAF.like,CXCL12,CXCL12,ITGB3,ITGB3,0.111111,0.108696,1.175410,1.279215,1.06552,0.535060,0.535060
1376,CAFs.MSC.iCAF.like,CAFs.MSC.iCAF.like,TGFB3,TGFB3,ACVRL1,ACVRL1,0.111111,0.111111,1.207899,1.224702,1.06552,0.533034,0.533034
831,Macrophage,Cancer.Her2.SC,CXCL12,CXCL12,CXCR4,CXCR4,0.105263,0.108696,1.199687,1.205923,1.06552,0.530261,0.530261
